In [27]:
from keras.models import load_model
from keras.preprocessing.image import (
    ImageDataGenerator,
    array_to_img,
    img_to_array,
    load_img,
)
import pandas as pd

DATAPATH = "data"
img_size = (300, 500)

# Test Model
model = load_model("temp-from-sc.keras")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 500, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 298, 498, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 296, 496, 64)      36928     
                                                                 
 batch_normalization (Batch  (None, 296, 496, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 148, 248, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 146, 246, 128)     73856 

In [28]:
val_set = pd.DataFrame()
val_set['filename'] = [f"{i+1}.jpg" for i in range(0, 300)]

In [29]:
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
validation_generator = val_datagen.flow_from_dataframe(
    val_set,
    directory=f"ranking_round/images/",
    x_col="filename",
    class_mode=None,
    shuffle=False,
    target_size=(img_size[0], img_size[1]),
)

Found 300 validated image filenames.


In [30]:
predict = model.predict(
    x=validation_generator,
    steps=len(validation_generator),
    workers=1,
    use_multiprocessing=False,
)

val_set = pd.DataFrame()
val_set['filename'] = [f"{i+1}.jpg" for i in range(0, 300)]
val_set["meat"] = predict[:, 0]
val_set["veggie"] = predict[:, 1]
val_set["noodle"] = predict[:, 2]

val_set

10/10 [==============================] - 2s 150ms/step


,filename,meat,veggie,noodle
0,1.jpg,11.315345,16.379486,104.845062
1,2.jpg,5.520959,14.136708,33.711849
2,3.jpg,3.886656,12.950157,1.822060
3,4.jpg,7.051526,13.963435,71.865044
4,5.jpg,5.344162,13.800299,47.526695
...,...,...,...,...
295,296.jpg,4.315170,13.399590,9.095552
296,297.jpg,4.891443,13.886817,19.051683
297,298.jpg,4.317421,13.309577,7.264389
298,299.jpg,3.889329,12.951249,1.880437


In [31]:
val_set.to_csv("result.csv", index=False)